In [1]:
import os
import flow
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import pandas as pd

In [22]:
mouse = 'OA27'
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

print(runs[1].run_type)
print(runs[1].tags)
run_tags = runs[1].tags
run_type = runs[1].run_type
hunger = [ 'hungry' for s in range(len(run_tags)) if run_tags[s] == 'hungry']
tags = [ s for s in range(len(run_tags)) if run_tags[s] != 'hungry' 
        and run_tags[s] != 'sated'
        and run_tags[s] != 'kelly'
        and run_tags[s] != run_type]
if tags == []:
    tags = 'normal'
ç

print(tags)

naive
(u'hungry', u'kelly', u'naive')
normal


In [39]:
print(run_type)
print(hunger)
print(tags)

naive
['hungry']
hungry


In [7]:
# tag_options = ['hungry', 'sated', 'disengaged', 'contrast', 'orientation_mapping', 'retinotopy']
# TODO add a seconds dataframe which is the trial variavles  

mouse = 'OA27'
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

# triggering parameters
start_time = -1
end_time = 6
trace_type = 'dff'

trial_list = []
for run in runs:
    
    # get your t2p object
    t2p = run.trace2p()
    
    # get your runtype and relevant tags 
    run_type = run.run_type
    run_tags = run.tags
    # get hunger state for all trials, consider hungry if not sated
    hunger = [ 'hungry' for s in range(len(run_tags)) if run_tags[s] == 'hungry']
    if hunger == []:
        hunger = [ 'hungry' for s in range(len(run_tags)) if run_tags[s] != 'sated']
    hunger = np.unique(hunger)[0]
    # get relevant trial-distinguising tags excluding kelly, hunger state, and run_type
    tags = [ run_tags[s] for s in range(len(run_tags)) if run_tags[s] != hunger 
            and run_tags[s] != 'kelly'
            and run_tags[s] != run_type]
    if tags == []: # define as "standard" if the run is not another option
        tags = ['standard']
    tags = tags[0]

    # get all of your trial variables of interest
    # use to index along axis=2 in cstraces/run_traces
    oris = []
    css = []
    trialerror = []
    for trial in t2p.d['condition']:
        
        # get cs and ori
        codename = t2p.d['codes'].keys()[t2p.d['codes'].values().index(t2p.d['condition'][trial])]
        oriname = t2p.d['orientations'][codename]
        css.append(codename)
        oris.append(oriname)
        
        # get trialerror
        trialerror.append(t2p.d['trialerror'][trial])

    oris = np.array(oris)

    # trigger all trials around stimulus onsets to get trial number
    run_traces = t2p.cstraces('', start_s=start_time, end_s=end_time, trace_type=trace_type,
                    cutoff_before_lick_ms=-1, errortrials=-1, baseline=None,
                    baseline_to_stimulus=True)

    # loop through and append each trial (slice of cstraces)
    for trial in range(np.shape(run_traces)[2]):
        index = pd.MultiIndex.from_arrays([
                    [run.mouse], 
                    [run.date],
                    [run.run],
                    [int(trial)]
                    ],
                    names=['mouse', 'date', 'run', 'trial_idx'])
                    
        data = {'orientation':  [oris[trial]], 'condition': [css[trial]],
                'trialerror': [trialerror[trial]], 'hunger': [hunger],
                'run_type': [run_type], 'tag': [tags]}

        # append all trials across all runs together in a list
#         trial_list.append(pd.DataFrame({'trial_idx': trial}, index=index))
        trial_list.append(pd.DataFrame(data,index=index))
    
    # clear your t2p to save RAM
    run._t2p = None
    print('Run: ' + str(run) + ': ' + str(len(trial_list)))

# concatenate all runs together in final dataframe 
trial_df = pd.concat(trial_list, axis=0)

# save
save_path = os.path.join(flow.paths.outd, str(runs[0].mouse) + '_df_klg_' + trace_type + '_trialmeta.pkl')
trial_df.to_pickle(save_path)

Run: OA27_170116_2: 220
Run: OA27_170116_3: 376
Run: OA27_170118_2: 596
Run: OA27_170118_3: 752
Run: OA27_170119_1: 907
Run: OA27_170119_2: 1059
Run: OA27_170125_2: 1280
Run: OA27_170125_3: 1435
Run: OA27_170125_4: 1547
Run: OA27_170127_2: 1702
Run: OA27_170127_3: 1857
Run: OA27_170128_2: 2012
Run: OA27_170128_3: 2167
Run: OA27_170128_4: 2306
Run: OA27_170129_2: 2453
Run: OA27_170129_3: 2600
Run: OA27_170129_4: 2741
Run: OA27_170129_5: 2884
Run: OA27_170131_4: 3026
Run: OA27_170131_5: 3167
Run: OA27_170131_6: 3308
Run: OA27_170201_3: 3455
Run: OA27_170201_4: 3592
Run: OA27_170201_5: 3729
Run: OA27_170201_6: 3864
Run: OA27_170201_7: 3995
Run: OA27_170202_2: 4129
Run: OA27_170202_3: 4262
Run: OA27_170202_4: 4391
Run: OA27_170202_5: 4518
Run: OA27_170204_2: 4661
Run: OA27_170204_4: 4803
Run: OA27_170204_5: 4935
Run: OA27_170204_6: 5064
Run: OA27_170206_3: 5215
Run: OA27_170206_4: 5358
Run: OA27_170206_5: 5513
Run: OA27_170207_3: 5680
Run: OA27_170207_4: 5833
Run: OA27_170207_5: 5993
Run: 

In [4]:
trial_df = pd.concat(trial_list, axis=0)
trial_df

Empty DataFrame
Columns: []
Index: [(OA27, 170116, 2, 0, 225, orientation_225, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 1, 135, orientation_135, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 2, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 3, 135, orientation_135, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 4, 315, orientation_315, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 5, 180, orientation_180, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 6, 270, orientation_270, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 7, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 8, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 9, 180, orientation_180, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 10, 45, orientation_45, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 11, 315, orientation_315, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 12, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 13, 45, orientation_45, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 14, 270, orientation_270, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 15, 225, orientation_225, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 16, 225, orientation_225, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 17, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 18, 45, orientation_45, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 19, 270, orientation_270, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 20, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 21, 135, orientation_135, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 22, 225, orientation_225, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 23, 135, orientation_135, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 24, 315, orientation_315, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 25, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 26, 180, orientation_180, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 27, 180, orientation_180, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 28, 270, orientation_270, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 29, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 30, 45, orientation_45, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 31, 315, orientation_315, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 32, 270, orientation_270, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 33, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 34, 225, orientation_225, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 35, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 36, 315, orientation_315, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 37, 270, orientation_270, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 38, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 39, 135, orientation_135, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 40, 225, orientation_225, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 41, 45, orientation_45, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 42, 45, orientation_45, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 43, 135, orientation_135, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 44, 180, orientation_180, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 45, 90, orientation_90, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 46, 180, orientation_180, 1, hungry, naive, orientation_mapping), (OA27, 170116, 2, 47, 315, orientation_315, 1, hungry, naive, 